In [675]:
## Importing libraries

In [676]:
import sklearn.utils._cython_blas


In [677]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from collections import Counter
from sklearn.datasets import make_classification

In [678]:
## Loading csv file containing historical sensor data into sensor_data dataframe

In [679]:
sensor_data=pd.read_csv('C:/Users/spatnam/Downloads/On-street_Car_Parking_Sensor_Data_-_2017.csv',nrows=50000)

In [680]:
Restriction_data=pd.read_csv('C:/Users/spatnam/Downloads/On-street_Car_Park_Bay_Restrictions.csv')

In [681]:
# Using shape function to know the shape of the dataset 

In [682]:
sensor_data.shape

(50000, 14)

In [683]:
Restriction_data.shape

(4263, 62)

In [684]:
sensor_data.head()

,DeviceId,ArrivalTime,DepartureTime,DurationSeconds,StreetMarker,Sign,Area,StreetId,StreetName,BetweenStreet1,BetweenStreet2,Side Of Street,In Violation,Vehicle Present
0,18773,04/11/2017 07:24:35 AM,04/11/2017 07:30:00 AM,325,2477S,NaN,Spencer,123,BOURKE STREET,SPENCER STREET,KING STREET,4,False,True
1,19262,07/03/2017 06:41:20 AM,07/03/2017 07:30:00 AM,2920,3598N,NaN,Magistrates,926,LITTLE LONSDALE STREET,SPENCER STREET,KING STREET,3,False,False
2,21322,04/07/2017 10:48:38 AM,04/07/2017 11:41:34 AM,3176,12548N,2P TKT A M-F 7:30-18:30,Jolimont,1413,WELLINGTON PARADE,JOLIMONT ROAD,CLARENDON STREET,3,False,True
3,17670,02/09/2017 04:57:38 PM,02/09/2017 05:11:04 PM,806,13836N,2P TKT A M-SAT 7:30-18:30,Docklands,915,LITTLE DOCKLANDS DRIVE,WATERFRONT WAY,FOOTSCRAY ROAD,3,False,True
4,19241,05/27/2017 10:43:52 PM,05/27/2017 10:47:28 PM,216,3571S,NaN,Magistrates,926,LITTLE LONSDALE STREET,SPENCER STREET,KING STREET,4,False,False


In [685]:
Restriction_data.head()

,BayID,DeviceID,Description1,Description2,Description3,Description4,Description5,Description6,DisabilityExt1,DisabilityExt2,...,ToDay3,ToDay4,ToDay5,ToDay6,TypeDesc1,TypeDesc2,TypeDesc3,TypeDesc4,TypeDesc5,TypeDesc6
0,1500,30574,LZ 30MINS 722 M-F 7.30 to 6.30,2P MTR M-F 18:30-20:30,2P MTR SAT 7:30-20:30,NaN,NaN,NaN,0,240.0,...,6.0,NaN,NaN,NaN,Loading Zone 30M 722,2P Meter,2P Meter,NaN,NaN,NaN
1,8634,28003,2P RPA 1B 7:30-23:00 M-SUN,NaN,NaN,NaN,NaN,NaN,240,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN
2,8587,27956,2P RPA 1B 7:30-23:00 M-SUN,NaN,NaN,NaN,NaN,NaN,240,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN
3,1499,30573,LZ 30MINS 722 M-F 7.30 to 6.30,2P MTR M-F 18:30-20:30,2P MTR SAT 7:30-20:30,NaN,NaN,NaN,0,240.0,...,6.0,NaN,NaN,NaN,Loading Zone 30M 722,2P Meter,2P Meter,NaN,NaN,NaN
4,8498,27876,2P RPA 1B 7:30-23:00 M-SUN,NaN,NaN,NaN,NaN,NaN,240,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN


In [686]:
sensor_data['Arr_Date'] = pd.to_datetime(sensor_data['ArrivalTime']).dt.date
sensor_data['Arr_Time'] = pd.to_datetime(sensor_data['ArrivalTime']).dt.time

In [687]:
sensor_data['Dep_Date'] = pd.to_datetime(sensor_data['DepartureTime']).dt.date
sensor_data['Dep_Time'] = pd.to_datetime(sensor_data['DepartureTime']).dt.time

In [688]:
sensor_data.drop('ArrivalTime',axis=1,inplace=True)
sensor_data.drop('DepartureTime',axis=1,inplace=True)

In [689]:
sensor_data['Vehicle Present'] = sensor_data['Vehicle Present'].map({False:0,True:1})
sensor_data['In Violation'] = sensor_data['In Violation'].map({False:0,True:1})

In [690]:
sensor_data[['StreetMarker','Area','StreetName','BetweenStreet1','BetweenStreet2']]= sensor_data[['StreetMarker','Area','StreetName','BetweenStreet1','BetweenStreet2']].apply(lambda x: pd.factorize(x)[0])

In [691]:
sensor_data.head()

,DeviceId,DurationSeconds,StreetMarker,Sign,Area,StreetId,StreetName,BetweenStreet1,BetweenStreet2,Side Of Street,In Violation,Vehicle Present,Arr_Date,Arr_Time,Dep_Date,Dep_Time
0,18773,325,0,NaN,0,123,0,0,0,4,0,1,2017-04-11,07:24:35,2017-04-11,07:30:00
1,19262,2920,1,NaN,1,926,1,0,0,3,0,0,2017-07-03,06:41:20,2017-07-03,07:30:00
2,21322,3176,2,2P TKT A M-F 7:30-18:30,2,1413,2,1,1,3,0,1,2017-04-07,10:48:38,2017-04-07,11:41:34
3,17670,806,3,2P TKT A M-SAT 7:30-18:30,3,915,3,2,2,3,0,1,2017-02-09,16:57:38,2017-02-09,17:11:04
4,19241,216,4,NaN,1,926,1,0,0,4,0,0,2017-05-27,22:43:52,2017-05-27,22:47:28


In [692]:
# Using info function to see the datatypes and number of instances in the dataset

In [693]:
sensor_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   DeviceId         50000 non-null  int64 
 1   DurationSeconds  50000 non-null  int64 
 2   StreetMarker     50000 non-null  int64 
 3   Sign             32703 non-null  object
 4   Area             50000 non-null  int64 
 5   StreetId         50000 non-null  int64 
 6   StreetName       50000 non-null  int64 
 7   BetweenStreet1   50000 non-null  int64 
 8   BetweenStreet2   50000 non-null  int64 
 9   Side Of Street   50000 non-null  int64 
 10  In Violation     50000 non-null  int64 
 11  Vehicle Present  50000 non-null  int64 
 12  Arr_Date         50000 non-null  object
 13  Arr_Time         50000 non-null  object
 14  Dep_Date         50000 non-null  object
 15  Dep_Time         50000 non-null  object
dtypes: int64(11), object(5)
memory usage: 6.1+ MB


In [694]:
Restriction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4263 entries, 0 to 4262
Data columns (total 62 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   BayID           4263 non-null   int64  
 1   DeviceID        4263 non-null   int64  
 2   Description1    4263 non-null   object 
 3   Description2    2862 non-null   object 
 4   Description3    584 non-null    object 
 5   Description4    217 non-null    object 
 6   Description5    82 non-null     object 
 7   Description6    27 non-null     object 
 8   DisabilityExt1  4263 non-null   int64  
 9   DisabilityExt2  2862 non-null   float64
 10  DisabilityExt3  584 non-null    float64
 11  DisabilityExt4  217 non-null    float64
 12  DisabilityExt5  82 non-null     float64
 13  DisabilityExt6  27 non-null     float64
 14  Duration1       4263 non-null   int64  
 15  Duration2       2862 non-null   float64
 16  Duration3       584 non-null    float64
 17  Duration4       217 non-null    f

In [695]:
## Data Cleaning

In [696]:
sensor_data.isna().sum()

DeviceId               0
DurationSeconds        0
StreetMarker           0
Sign               17297
Area                   0
StreetId               0
StreetName             0
BetweenStreet1         0
BetweenStreet2         0
Side Of Street         0
In Violation           0
Vehicle Present        0
Arr_Date               0
Arr_Time               0
Dep_Date               0
Dep_Time               0
dtype: int64

In [697]:
Restriction_data.isna().sum()

BayID              0
DeviceID           0
Description1       0
Description2    1401
Description3    3679
                ... 
TypeDesc2       1401
TypeDesc3       3679
TypeDesc4       4046
TypeDesc5       4181
TypeDesc6       4236
Length: 62, dtype: int64

In [698]:
sensor_data=sensor_data.select_dtypes(exclude=['object'])
Restriction_data=Restriction_data.select_dtypes(exclude=['object'])

In [699]:
Bay_data = sensor_data.merge(Restriction_data,left_on='DeviceId', right_on='DeviceID',how='outer')

In [700]:
Bay_data.shape

(54116, 44)

In [701]:
Bay_data.head()

,DeviceId,DurationSeconds,StreetMarker,Area,StreetId,StreetName,BetweenStreet1,BetweenStreet2,Side Of Street,In Violation,...,FromDay3,FromDay4,FromDay5,FromDay6,ToDay1,ToDay2,ToDay3,ToDay4,ToDay5,ToDay6
0,18773.0,325.0,0.0,0.0,123.0,0.0,0.0,0.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18773.0,60.0,0.0,0.0,123.0,0.0,0.0,0.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18773.0,27000.0,0.0,0.0,123.0,0.0,0.0,0.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18773.0,258.0,0.0,0.0,123.0,0.0,0.0,0.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18773.0,702.0,0.0,0.0,123.0,0.0,0.0,0.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [702]:
Bay_data.shape

(54116, 44)

In [703]:
null_val=Bay_data.isnull().sum()
toRemove =set()

In [704]:
null_val

DeviceId            4116
DurationSeconds     4116
StreetMarker        4116
Area                4116
StreetId            4116
StreetName          4116
BetweenStreet1      4116
BetweenStreet2      4116
Side Of Street      4116
In Violation        4116
Vehicle Present     4116
BayID              48993
DeviceID           48993
DisabilityExt1     48993
DisabilityExt2     50969
DisabilityExt3     53465
DisabilityExt4     53835
DisabilityExt5     54021
DisabilityExt6     54076
Duration1          48993
Duration2          50969
Duration3          53465
Duration4          53835
Duration5          54021
Duration6          54076
EffectiveOnPH1     48993
EffectiveOnPH2     50969
EffectiveOnPH3     53465
EffectiveOnPH4     53835
EffectiveOnPH5     54021
EffectiveOnPH6     54076
Exemption6         54116
FromDay1           48993
FromDay2           50969
FromDay3           53465
FromDay4           53835
FromDay5           54021
FromDay6           54076
ToDay1             48993
ToDay2             50969


In [705]:
for i in null_val.index:
    if(null_val[i]>500):
        toRemove.add(i)
for i in toRemove:
    Bay_data.drop(i,axis=1,inplace=True)
    

In [706]:
#separating dependant and independant features

In [707]:
x = Bay_data.drop(['In Violation'], axis=1)
y = Bay_data['In Violation']

KeyError: "['In Violation'] not found in axis"

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
y.value_counts()

In [ ]:
## Visualising features

In [ ]:
y.value_counts().plot.pie(autopct='%.2f')

In [ ]:
#Loading library for splitting the data
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.30,shuffle=True, random_state=101)

In [ ]:
print("x-train - ",x_train.shape)
print("y-train - ",y_train.shape)
print("x-test - ",x_test.shape)
print("y-test - ",y_test.shape)

In [ ]:
# This shows that the dataset is significantly imbalanced

In [ ]:
#Logistic Regression with imbalanced data

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg=LogisticRegression()
logreg.fit(x_train,y_train)

In [ ]:
y_pred=logreg.predict(x_test)

In [ ]:
#Model Evaluation

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


In [ ]:
print(metrics.classification_report(y_test,y_pred))